In [1]:
# VGG6 实例化
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150,150,3))

Using TensorFlow backend.






58892288/58889256 [==============================] - 3s 0us/step








In [2]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
%cd drive/My\ Drive/keras_data/dogs-vs-cats

/content/drive/My Drive/keras_data/dogs-vs-cats


In [0]:
# 在顶部添加Dense层来扩展已用模型
# 在卷积基上添加一个密集连接分类器
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [0]:
# 冻结
conv_base.trainable = False

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

train_dir = 'train'
validation_dir = 'validation'
test_dir = 'test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary'
)

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50
)

In [0]:
# 冻结直到某一层的所有层
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [0]:
# 微调模型
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=50
)

Epoch 1/100
100/100 [==============================] - 160s 2s/step - loss: 0.4236 - acc: 0.8080 - val_loss: 0.3058 - val_acc: 0.8640
Epoch 2/100
100/100 [==============================] - 22s 224ms/step - loss: 0.3298 - acc: 0.8540 - val_loss: 0.2331 - val_acc: 0.8980
Epoch 3/100
100/100 [==============================] - 22s 223ms/step - loss: 0.3121 - acc: 0.8650 - val_loss: 0.2106 - val_acc: 0.9150
Epoch 4/100
100/100 [==============================] - 22s 224ms/step - loss: 0.2767 - acc: 0.8845 - val_loss: 0.2363 - val_acc: 0.8970
Epoch 5/100
100/100 [==============================] - 23s 226ms/step - loss: 0.2567 - acc: 0.8925 - val_loss: 0.1883 - val_acc: 0.9200
Epoch 6/100
100/100 [==============================] - 22s 224ms/step - loss: 0.2307 - acc: 0.9085 - val_loss: 0.1852 - val_acc: 0.9240
Epoch 7/100
100/100 [==============================] - 22s 224ms/step - loss: 0.2285 - acc: 0.9045 - val_loss: 0.2015 - val_acc: 0.9190
Epoch 8/100
100/100 [=============================